In [1]:
from llama_cpp import Llama
import re
import json
from IPython.core.display import display, HTML
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler


callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

/var/folders/_b/ccc66ybs0m59z0yyhly4_jp80000gn/T/ipykernel_4922/661250872.py:4: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
llm = Llama(model_path="/Users/ananyahooda/.cache/lm-studio/models/TheBloke/Mistral-7B-Instruct-v0.2-GGUF/mistral-7b-instruct-v0.2.Q4_K_S.gguf",  
n_ctx=2048,
n_gpu_layers=-1,
n_batch=512,
callback_manager=callback_manager,
verbose=True,)

llama_model_loader: loaded meta data with 24 key-value pairs and 291 tensors from /Users/ananyahooda/.cache/lm-studio/models/TheBloke/Mistral-7B-Instruct-v0.2-GGUF/mistral-7b-instruct-v0.2.Q4_K_S.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.2
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dime

In [ ]:
question = "Please extract"

In [ ]:
output = llm(question, max_tokens=100)

In [ ]:
# The available action to Assistant is:
# - "extract_text_triplets": Useful for when Assistant is asked to extract triplets from a given text.
#   - To use the extract_triplets tool, Assistant should respond like so:
#     {{"action": "extract_text_triplets", "action_input": "Your text here"}}

# Here are some previous conversations between the Assistant and User:

# User: Hey how are you today?
# Assistant: I'm good thanks, how are you?
# User: Can you extract all the triplets from this text: "Gràcia is a district of the city of Barcelona, Spain."
# Assistant: {{"action": "extract_text_triplets", "action_input": "Gràcia is a district of the city of Barcelona, Spain."}}
# User: Also give triples for "obama was US president"
# Assistant: {{"action": "extract_text_triplets", "action_input": "obama was US president"}}




In [83]:
prompt_template = '''<s>[INST] <<SYS>>
Assistant is an expert JSON builder designed to assist with a tasks of Triple extraction from text.

Assistant is able to extract triples from the text.

Only type on relation available are 5 realtions: {{'employer', 'headquarters location', 'killed by', 'location', 'residence'}}

Only type of entities are: {{'People', 'Organisation', 'Location', 'Other'}}

Assistant returns the response as the list of dictionaries only always and dictionaries consisting of keys called "head", "type" and "tail" in ever dictionary in the list

Here are some previous conversations between the Assistant and User:

User: Hey how are you today?
Assistant: I'm good thanks, how are you?
User: Can you extract all the triplets from this text: John Wilkes Booth , who assassinated President Lincoln , was an actor .
Assistant:  [{{"head": "President Lincoln", "type": "killed by", "tail": "John Wilkes Booth"}}]
User: Also give triples for text: "Marie Magdefrau Ferraro , 50 , of Bethany , Conn. , was shot to death Thursday when two bandits armed with assault rifles emerged from nearby bushes and began firing at a van carrying a Connecticut Audubon Society wildlife wild tour group .
Assistant: [{{"head": "Marie Magdefrau Ferraro", "type": "residence", "tail": "Bethany"}}, {{"head": "Marie Magdefrau Ferraro", "type": "residence", "tail": "Conn." }},{{"head": "Bethany", "type": "location", "tail": "Conn."}}] 
User: Thanks, Bye!
Assistant: See you later, the Chat is closed.
<</SYS>>

{0}[/INST]'''

In [69]:
ques = "Please extract triples for: In the field of mechanics , Wang Ziqiang at the Institute of Mechanics has made considerable headway in the area of elastoplastic crack mechanics ."

In [48]:
import ast

In [77]:
def process_command1(command):
    # Put user command into prompt
    prompt = prompt_template.format("User: " + command)
    # Send command to the model
    output = llm(prompt, max_tokens=2000, stop=["User:"])
    response = (output['choices'][0]['text']).strip()
    response = ast.literal_eval(response)
    return response

In [84]:
process_command1("Please extract triples for: In the field of mechanics , Wang Ziqiang at the Institute of Mechanics has made considerable headway in the area of elastoplastic crack mechanics .")

Llama.generate: prefix-match hit

llama_print_timings:        load time =    8692.62 ms
llama_print_timings:      sample time =      25.50 ms /    85 runs   (    0.30 ms per token,  3332.94 tokens per second)
llama_print_timings: prompt eval time =    4083.89 ms /   355 tokens (   11.50 ms per token,    86.93 tokens per second)
llama_print_timings:        eval time =    6740.42 ms /    84 runs   (   80.24 ms per token,    12.46 tokens per second)
llama_print_timings:       total time =   11306.71 ms /   439 tokens


[{'head': 'Wang Ziqiang',
  'type': 'headquarters location',
  'tail': 'Institute of Mechanics'},
 {'head': 'Wang Ziqiang',
  'type': 'employer',
  'tail': 'Institute of Mechanics'},
 {'head': 'Institute of Mechanics',
  'type': 'location',
  'tail': 'field of mechanics'}]

In [85]:
process_command1("Extract triples for: John Wilkes Booth , who assassinated President Lincoln , was an actor .")

Llama.generate: prefix-match hit

llama_print_timings:        load time =    8692.62 ms
llama_print_timings:      sample time =      10.56 ms /    55 runs   (    0.19 ms per token,  5209.81 tokens per second)
llama_print_timings: prompt eval time =     422.58 ms /    26 tokens (   16.25 ms per token,    61.53 tokens per second)
llama_print_timings:        eval time =    4617.62 ms /    54 runs   (   85.51 ms per token,    11.69 tokens per second)
llama_print_timings:       total time =    5232.67 ms /    80 tokens


[{'head': 'President Lincoln',
  'type': 'killed by',
  'tail': 'John Wilkes Booth'},
 {'head': 'John Wilkes Booth', 'type': 'profession', 'tail': 'actor'}]

In [89]:
import json

# Function to extract triples for each context in eval.json and create pred.json
def generate_pred_json(eval_file_path, pred_file_path):
    # Load the evaluation data from eval.json
    with open(eval_file_path, 'r') as file:
        eval_data = json.load(file)
    
    # Initialize a list to hold the modified data with extracted triples
    modified_data = []
    
    # Iterate over each item in the evaluation data
    for item in eval_data:
        context = item['context']
        # Prepare the command with the context
        command = f"Can you extract all the triplets from this text: {context}"
        # Use the process_command function to predict the extracted triples
        try:
            extracted_triplets = process_command1(command)
            item['triples'] = extracted_triplets
        # Append the modified item to the modified_data list
            modified_data.append(item)
        except SyntaxError as e:
            print(f"Caught a SyntaxError: {e}")
        # Append the extracted triples to the item under the 'triples' key
       
    
    # Write the modified data with extracted triples to pred.json
    with open(pred_file_path, 'w') as file:
        json.dump(modified_data, file, indent=4)

# Example usage
eval_file_path = '/Users/ananyahooda/Desktop/final/data/extracted_data/conll04/conll04_small.json' # Replace with the actual path to your eval.json file
pred_file_path = '/Users/ananyahooda/Desktop/final/small_pred_conll04.json' # The output file path
generate_pred_json(eval_file_path, pred_file_path)

Llama.generate: prefix-match hit

llama_print_timings:        load time =    8692.62 ms
llama_print_timings:      sample time =      44.57 ms /   160 runs   (    0.28 ms per token,  3589.70 tokens per second)
llama_print_timings: prompt eval time =     891.24 ms /    44 tokens (   20.26 ms per token,    49.37 tokens per second)
llama_print_timings:        eval time =   12858.51 ms /   159 runs   (   80.87 ms per token,    12.37 tokens per second)
llama_print_timings:       total time =   14693.93 ms /   203 tokens
Llama.generate: prefix-match hit


Caught a SyntaxError: unexpected indent (<unknown>, line 2)



llama_print_timings:        load time =    8692.62 ms
llama_print_timings:      sample time =     109.99 ms /   363 runs   (    0.30 ms per token,  3300.33 tokens per second)
llama_print_timings: prompt eval time =     577.50 ms /    37 tokens (   15.61 ms per token,    64.07 tokens per second)
llama_print_timings:        eval time =   29238.47 ms /   362 runs   (   80.77 ms per token,    12.38 tokens per second)
llama_print_timings:       total time =   32182.37 ms /   399 tokens
Llama.generate: prefix-match hit


Caught a SyntaxError: EOL while scanning string literal (<unknown>, line 1)



llama_print_timings:        load time =    8692.62 ms
llama_print_timings:      sample time =      45.94 ms /   159 runs   (    0.29 ms per token,  3461.04 tokens per second)
llama_print_timings: prompt eval time =     315.29 ms /    23 tokens (   13.71 ms per token,    72.95 tokens per second)
llama_print_timings:        eval time =   12582.45 ms /   158 runs   (   79.64 ms per token,    12.56 tokens per second)
llama_print_timings:       total time =   13864.38 ms /   181 tokens
Llama.generate: prefix-match hit


...

Caught a SyntaxError: invalid syntax (<unknown>, line 1)



llama_print_timings:        load time =    8692.62 ms
llama_print_timings:      sample time =      48.31 ms /   104 runs   (    0.46 ms per token,  2152.81 tokens per second)
llama_print_timings: prompt eval time =     585.27 ms /    39 tokens (   15.01 ms per token,    66.64 tokens per second)
llama_print_timings:        eval time =    8825.88 ms /   103 runs   (   85.69 ms per token,    11.67 tokens per second)
llama_print_timings:       total time =   10156.89 ms /   142 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =    8692.62 ms
llama_print_timings:      sample time =      24.91 ms /    49 runs   (    0.51 ms per token,  1966.84 tokens per second)
llama_print_timings: prompt eval time =     592.18 ms /    45 tokens (   13.16 ms per token,    75.99 tokens per second)
llama_print_timings:        eval time =    4001.07 ms /    48 runs   (   83.36 ms per token,    12.00 tokens per second)
llama_print_timings:       total time =    4939.45 ms /    93 

Caught a SyntaxError: invalid syntax (<unknown>, line 1)



llama_print_timings:        load time =    8692.62 ms
llama_print_timings:      sample time =      15.84 ms /    31 runs   (    0.51 ms per token,  1957.19 tokens per second)
llama_print_timings: prompt eval time =     320.68 ms /    16 tokens (   20.04 ms per token,    49.89 tokens per second)
llama_print_timings:        eval time =    2630.56 ms /    30 runs   (   87.69 ms per token,    11.40 tokens per second)
llama_print_timings:       total time =    3174.15 ms /    46 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =    8692.62 ms
llama_print_timings:      sample time =      39.97 ms /    96 runs   (    0.42 ms per token,  2402.04 tokens per second)
llama_print_timings: prompt eval time =     584.64 ms /    38 tokens (   15.39 ms per token,    65.00 tokens per second)
llama_print_timings:        eval time =    7766.43 ms /    95 runs   (   81.75 ms per token,    12.23 tokens per second)
llama_print_timings:       total time =    8986.35 ms /   133 

Caught a SyntaxError: invalid syntax (<unknown>, line 1)
